In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import requests
import warnings
warnings.simplefilter('ignore')
import json
import cfscrape
import time
import datetime



def get_sourse_html(url):
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.maximize_window()
    try:
        driver.get(url=url)
        time.sleep(3)

        # while True:
        try:
            driver.find_element(By.CLASS_NAME, "input__control._bold").send_keys('Таттелеком')
        except Exception as ex:
            print (ex)

        try:
            search_button = driver.find_element(By.CLASS_NAME, 'small-search-form-view__button')
            zoom_out = driver.find_element(By.CLASS_NAME, 'zoom-control__zoom-out')
            actions = ActionChains(driver)
            actions.click(search_button).perform()
            time.sleep(1)
            actions.click(zoom_out).perform()
            time.sleep(3)
            actions.click(zoom_out).perform()
            time.sleep(1)
            actions.click(zoom_out).perform()
        except Exception as ex:
            print (ex)
        time.sleep(5)

        while True:
            if driver.find_elements(By.CLASS_NAME, "add-business-view"):
                find_more_element = driver.find_element(By.CLASS_NAME, "scroll__scrollbar-thumb")
                actions2 = ActionChains(driver)
                actions2.drag_and_drop_by_offset(find_more_element, 0, 100).perform()
                time.sleep(3)
                actions2.drag_and_drop_by_offset(find_more_element, 0, 100).perform()
                time.sleep(3)
                actions2.drag_and_drop_by_offset(find_more_element, 0, 100).perform()
                time.sleep(3)
                with open("test.html", "w", encoding = "utf-8") as file:
                    file.write(driver.page_source)

                break
            else:
                find_more_element = driver.find_element(By.CLASS_NAME, "scroll__scrollbar-thumb")
                try:
                    actions2 = ActionChains(driver)
                    actions2.drag_and_drop_by_offset(find_more_element, 0, 100).perform()
                    time.sleep(3)
                except Exception as ex:
                    print (0)
    except Exception as ex:
        print (ex)
    finally:
        driver.close()
        driver.quit()
get_sourse_html(url = 'https://yandex.ru/maps/?ll=52.583698%2C56.067155&z=5.86')
time.sleep(30)

ModuleNotFoundError: No module named 'cfscrape'

In [123]:
import pandas as pd
from bs4 import BeautifulSoup
arr = []
def parsing_data():
    test = open('test.html', encoding="utf-8")
    soup = BeautifulSoup(test, 'lxml')
    try:
        reviews = soup.find_all('li', {"class": "search-snippet-view"})
    except:
        return None
    for i in range(len(reviews)):
        id_filials = reviews[i].find('div', {"class": 'search-snippet-view__body _type_business'})
        id_filials = id_filials['data-id']
        arr.append(id_filials)
def main():
    parsing_data()
if __name__ == "__main__":
    main()

TypeError: 'NoneType' object is not subscriptable

In [113]:
Data_id = pd.DataFrame(arr, columns=['id'])

In [115]:
Data_id.head(35)

,id
0,5651209305
1,1315558370
2,66697421636
3,1230403863
4,106396488794
5,68938452823
6,13084226257
7,217576127302
8,1249340350
9,1080903330


In [106]:
len(Data_id)

35

In [116]:
estimates_all = pd.DataFrame()

In [117]:
for i in range(len(Data_id)):
    yandex_id = str(Data_id.iloc[i]['id'])
    date_object = time.strftime("%d.%m.%Y")
    time_object = time.strftime("%H:%M:%S")
    ya = pd.DataFrame({"date": date_object,"time":time_object, "id": yandex_id}, index = [1])
    ya['key'] = 1
    def get_count_star(review_stars):
        star_count = 0
        for review_star in review_stars:
            if '_empty' in review_star.get('class'):
                continue
            elif '_half' in review_star.get('class'):
                star_count = star_count + 0.5
            else:
                star_count = star_count + 1
        return star_count

    def get_session():
        session = requests.Session()
        session.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
            'upgrade-insecure-requests': '1',
            #'user-agent': useragent.random,
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
            'viewport-width': '1920'
        }
        return cfscrape.create_scraper(sess=session)

    def parsing_data(data_result, r):
        soup = BeautifulSoup(r.text, 'html.parser')
        try:
            reviews = soup.find('div', {"data-chunk": "reviews"})
        except:
            return None
        try:
            summary_rating__main = reviews.find('div', {"class": "business-summary-rating__main-rating"})
        except:
            summary_rating__main = None
        try:
            rank_summary_rating__main = summary_rating__main.find('div',{"class": "business-summary-rating-badge-view__rating"}).text
        except:
            rank_summary_rating__main = None
        try:
            count_summary_rating__main = summary_rating__main.find('div', {
                "class": "business-summary-rating-badge-view__rating-count"}).text
        except:
            count_summary_rating__main = None
        try:
            stars_summary_rating__main = get_count_star(
                summary_rating__main.find('div', {"class": "business-rating-badge-view__stars"}))
        except:
            stars_summary_rating__main = None
        try:
            address = soup.find('meta', {"itemprop": "address"}).get('content')
        except:
            address = None
        
        try:
            name = soup.find('h1',{"class":"orgpage-header-view__header","itemprop":"name"}).text
        except:
            return None   
        
        data_result['company_info'] = {
            'company_rating': rank_summary_rating__main,
            'company_count_reviews': count_summary_rating__main,
            'company_count_stars': stars_summary_rating__main,
            'company_address': address,
            'company_name': name
        }
        return data_result
    
    url = 'https://yandex.ru/maps/org/letay/' + yandex_id
    session = get_session()
    r = session.get(url)
    data_result = {
        'company_info': {}
    }
    data_result
    data_result = parsing_data(data_result, r)
    df = pd.DataFrame(data_result)
    addr = df.loc['company_address']['company_info']
    name = df.loc['company_name']['company_info']
    while addr == None:
        time_new = time.strftime("%H:%M:%S")
        print(time_new,' New query for ', name, url)
        time.sleep(300)
        session = get_session()
        r = session.get(url)
        data_result = {
            'company_info': {}
        }
        data_result
        data_result = parsing_data(data_result, r)
        df = pd.DataFrame(data_result)
        addr = df.loc['company_address']['company_info']
    df = df.transpose()   
    if len(data_result['company_info']) == 0:
        data_result = None
    with open('company_reviews.json', 'w', encoding='utf-8') as f:
        json.dump(data_result, f, ensure_ascii=False, indent=4)
    print(i,' ',time_object,' ', name, ' ID:',yandex_id,' ',url,' - Success')
    df['key'] = 1
    df['url'] = url
    df = ya.merge(df, on = 'key', indicator=True)
    estimates_all = pd.concat([estimates_all,df])
   # time.sleep(7+i)

KeyError: 'company_address'

In [396]:
estimates_all

,date,time,id,key,company_address,company_count_reviews,company_count_stars,company_name,company_rating,url,_merge
0,23.11.2022,19:25:53,5651209305,1,"Республика Татарстан, Нижнекамский район, посе...",64 оценки,4,Летай,"Рейтинг 4,2",https://yandex.ru/maps/org/letay/5651209305,both
0,23.11.2022,19:25:53,1315558370,1,"Республика Татарстан, Набережные Челны, проспе...",423 оценки,2.5,Таттелеком Летай,"Рейтинг 2,6",https://yandex.ru/maps/org/letay/1315558370,both
0,23.11.2022,19:25:53,66697421636,1,"Республика Татарстан, Нижнекамск, проспект Хим...",31 оценка,4.5,Таттелеком,"Рейтинг 4,3",https://yandex.ru/maps/org/letay/66697421636,both
0,23.11.2022,19:25:54,1230403863,1,"Республика Татарстан, Альметьевск, улица Маяко...",367 оценок,4,Таттелеком,"Рейтинг 3,8",https://yandex.ru/maps/org/letay/1230403863,both
0,23.11.2022,19:25:54,106396488794,1,"Республика Татарстан, Нижнекамск, Корабельная ...",72 оценки,3.5,Таттелеком,"Рейтинг 3,7",https://yandex.ru/maps/org/letay/106396488794,both
0,23.11.2022,19:25:55,68938452823,1,"Республика Татарстан, Набережные Челны, проспе...",12 оценок,4,Таттелеком,"Рейтинг 3,9",https://yandex.ru/maps/org/letay/68938452823,both
0,23.11.2022,19:25:55,13084226257,1,"Республика Татарстан, Мамадыш, улица Тукая, 32",7 оценок,4,Летай,"Рейтинг 4,1",https://yandex.ru/maps/org/letay/13084226257,both
0,23.11.2022,19:25:55,217576127302,1,"Республика Татарстан, Нижнекамск, улица Баки У...",None,None,Таттелеком,None,https://yandex.ru/maps/org/letay/217576127302,both
0,23.11.2022,19:25:56,1249340350,1,"Республика Татарстан, Казань, улица Восстания,...",297 оценок,3,Таттелеком,"Рейтинг 3,1",https://yandex.ru/maps/org/letay/1249340350,both
0,23.11.2022,19:25:56,1080903330,1,"Республика Татарстан, Чистополь, улица Ленина,...",191 оценка,4,Таттелеком,"Рейтинг 4,2",https://yandex.ru/maps/org/letay/1080903330,both


In [308]:
day = time.strftime("%d%m%Y_%H%M")

In [309]:
estimates_all.to_excel('D:\\estimates\estimates_all_'+day+'.xlsx',index = False)